# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import nltk
from nltk.corpus import stopwords
nltk.download(['punkt', 'wordnet','stopwords'])

In [ ]:
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn import preprocessing
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


from sqlalchemy import create_engine

In [ ]:
# load data from database
engine = create_engine('sqlite:///MessageDatabase.db')
df = pd.read_sql_table("MessagesTable", con=engine)
X = df.message.values
#y = df[df.columns[4:]].values
#y = preprocessing.label_binarize(y, classes=[range(0,36)])
y = df.loc[:,"related":].values 

In [ ]:
labels = df.columns[4:]

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    #normalize - Remove punctuation characters
    regex = '[^a-zA-Z0-9]'
    detect_text = re.findall(regex, text)

    #tokenize - split text into words
    for list in text:
        list = text.replace(regex," ")
        tokens = word_tokenize(list)

    # Remove Stopwords
    words = [word for word in tokens if word not in stopwords.words('english')]

    #lemanize
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []

    for word in words:
        clean_tok = lemmatizer.lemmatize(word).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
# build pipeline
forest = RandomForestClassifier()

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(forest))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, target_names=labels))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
#Test
import time

parameters_short = {
    'clf__estimator__n_estimators': [10],
    'vect__ngram_range': [(1, 1)]
}

start = time.process_time()

cv = GridSearchCV(estimator=pipeline, param_grid=parameters_short, scoring='f1_macro', cv=None, n_jobs=-1,verbose=10)
cv.fit(X_train, Y_train)

print(time.process_time() - start)

In [ ]:
parameters = {
    'clf__estimator__n_estimators': [20],
    'vect__ngram_range': [(1, 1)]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, n_jobs=-1, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
def load_data():
    engine = create_engine('sqlite:///MessageDatabase.db')
    df = pd.read_sql_table("MessagesTable", con=engine)
    X = df.message.values
    #y = df[df.columns[4:]].values
    #y = preprocessing.label_binarize(y, classes=[range(0,36)])
    y = df.loc[:,"related":].values 
    return X, y


def display_results(cv, y_test, y_pred):
    labels = df.columns[4:]
    classification_report = classification_report(y_test, y_pred, target_names=labels)
    
    print("classification_report:\n", classification_report)
    print("\nBest Parameters:", cv.best_params_)


def main(model):
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    display_results(model, y_test, y_pred)


main(cv)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.